In [1]:
!pip  install transformers -q

In [2]:
import numpy as np
import pandas as pd

import json

import transformers


from transformers import DistilBertTokenizer
# from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

import tensorflow as tf

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [3]:
with open('generated_data.json') as file:
  data = json.loads(file.read())

In [4]:
data = data['data']

In [5]:
data = np.array(data)

In [6]:
data = pd.DataFrame(data, columns = ['phrases', 'intents'])

In [7]:
data['encoded_intents'] = data['intents'].astype('category').cat.codes

In [8]:
print(data)

                            phrases intents  encoded_intents
0                         call alex    call                0
1                  please call mark    call                0
2               i want to call mark    call                0
3                  let me call mark    call                0
4                  can we call mark    call                0
...                             ...     ...              ...
1345       what is the current time    time                2
1346        what's the current time    time                2
1347  what time is it in california    time                2
1348        please give me the time    time                2
1349               the time is what    time                2

[1350 rows x 3 columns]


In [9]:
data_texts = data['phrases'].to_list()

data_labels = data['encoded_intents'].to_list()

In [10]:
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.2, random_state = 0 )


# train_texts, test_texts, train_labels, test_labels = train_test_split(data_texts, train_labels, test_size = 0.2, random_state = 0 )

In [17]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation = True, padding = True, max_length = 500, return_tensors = "tf")

val_encodings = tokenizer(val_texts, truncation = True, padding = True,  max_length = 500, return_tensors = "tf" )

# test_encodings = tokenizer(test_texts, truncation = True, padding = True,  max_length = 500 )

In [18]:
train_encodings

{'input_ids': <tf.Tensor: shape=(1080, 15), dtype=int32, numpy=
array([[  101, 13764,  1060, ...,     0,     0,     0],
       [  101,  3531, 13764, ...,     0,     0,     0],
       [  101,  2064,  2017, ...,     0,     0,     0],
       ...,
       [  101,  2129,  2397, ...,     0,     0,     0],
       [  101,  2064,  2017, ...,     0,     0,     0],
       [  101,  2173,  1037, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1080, 15), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))


val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

# test_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(test_encodings),
#     test_labels
# ))

In [13]:
train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(15,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(15,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [19]:
from transformers import TrainingArguments, Trainer, TFDistilBertForSequenceClassification

training_args = TrainingArguments(output_dir="./result", evaluation_strategy="epoch")

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 3 )

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [20]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, metrics=['accuracy'])

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [21]:
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/3
68/68 [==============================] - 224s 3s/step - loss: 0.4285 - accuracy: 0.8380 - val_loss: 0.1109 - val_accuracy: 0.9704
Epoch 2/3
68/68 [==============================] - 180s 3s/step - loss: 0.1792 - accuracy: 0.9250 - val_loss: 0.0843 - val_accuracy: 0.9667
Epoch 3/3
68/68 [==============================] - 177s 3s/step - loss: 0.1466 - accuracy: 0.9361 - val_loss: 0.0790 - val_accuracy: 0.9704


In [45]:
texts = [
          "make a call to rohit",
          "please text rohit",
          "get card details",
          "connect",
          "send an email to rohit",
          "help",
          "call and chat",
          "chat and call",
          "email someone",
          "get time from india",
         "call rohit", 
         "call and chat", 
         "chat and call", 
         "help", 
         "connect with rohit", 
         "dial rohit", 
         "texting rohit", 
         "make a call and chat", 
         "connect me with helpdesk",
         "onboard me"
        ]

for text in texts:

  predict_input = tokenizer.encode(text,
                                  truncation=True,
                                  padding=True,
                                  return_tensors="tf")
  tf_output = model.predict(predict_input)[0]
  print(text)
  print(tf_output)
  tf_prediction = tf.nn.sigmoid(tf_output).numpy()[0]
  for prediction in tf_prediction:
    print(prediction)
  print("======================")

1/1 [==============================] - 0s 145ms/step
make a call to rohit
[[ 3.832683 -1.590358 -3.149823]]
0.9788074
0.16933352
0.041098256
1/1 [==============================] - 0s 112ms/step
please text rohit
[[-2.0426817  3.9051764 -3.068287 ]]
0.11479395
0.98026013
0.044434506
1/1 [==============================] - 0s 95ms/step
get card details
[[ 2.6885598   0.17730501 -3.756939  ]]
0.9363482
0.5442105
0.02282211
1/1 [==============================] - 0s 135ms/step
connect
[[ 1.9757013  0.6664451 -3.4063187]]
0.87822217
0.6607067
0.032098573
1/1 [==============================] - 0s 171ms/step
send an email to rohit
[[ 0.417647   2.5967636 -4.0802903]]
0.6029201
0.930653
0.01662161
1/1 [==============================] - 0s 129ms/step
help
[[ 2.3715224   0.49485078 -3.6090202 ]]
0.9146298
0.6212484
0.026364457
1/1 [==============================] - 0s 147ms/step
call and chat
[[-2.0949383  3.8443038 -2.947707 ]]
0.10958976
0.9790471
0.049845003
1/1 [==============================]